# 원, 삼각형, 사각형 구분하기 

In [1]:
import numpy as np 
import pandas as pd
import tensorflow as tf

seed = 2021
np.random.seed(seed)
tf.random.set_seed(seed)

### 데이터 셋 부풀리기  
- 어떻게 진행이 되는지 보기 : preview 폴더에 저장됨


In [2]:
from tensorflow.keras.preprocessing import image

train_datagen = image.ImageDataGenerator(
    rescale = 1/255.,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.5,
    zoom_range=[0.8,2.0],
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
    )

In [3]:
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img
img = load_img('hard_handwriting_shape/train/triangle/triangle001.png')
x = img_to_array(img)
x = x.reshape((1,) + x.shape)

In [4]:
i = 0
for batch in train_datagen.flow(x,batch_size=1, save_to_dir='preview',save_prefix='tri',save_format='png'):
    i += 1
    if i > 30:
        break


- train의 경우 

In [5]:
# 서브 디렉토리별로 데이터를 가져와서 
train_generator = train_datagen.flow_from_directory(
    'hard_handwriting_shape/train', 
    target_size=(24,24),                    # 이미지의 속성 - 자세히 가서 픽셀 사이즈 확인
    batch_size=3,
    class_mode='categorical'
)

Found 45 images belonging to 3 classes.


- test의 경우 

In [6]:
test_datagen = image.ImageDataGenerator(rescale = 1/255.)           # 데이터 셋을 0과 1사이로 만들어주는 것

# 서브 디렉토리별로 데이터를 가져와서 
test_generator = test_datagen.flow_from_directory(
    'hard_handwriting_shape/test', 
    target_size=(24,24),
    batch_size=3,
    class_mode='categorical'
)

Found 15 images belonging to 3 classes.


In [7]:
train_generator.labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2])

In [8]:
train_generator.filenames[0]

'circle\\circle001.png'

### 모델 정의 / 설정 / 학습

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [10]:
model = Sequential([
    Conv2D(32,(3,3), input_shape=(24,24,3),activation = 'relu'),
    Conv2D(64,(3,3),activation='relu'),

    MaxPooling2D(),
    Flatten(),
    
    Dense(128,activation='relu'),
    Dense(3, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 22, 22, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 10, 10, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6400)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               819328    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 387       
Total params: 839,107
Trainable params: 839,107
Non-trainable params: 0
__________________________________________________

In [11]:
model.compile(
    optimizer='adam',
    loss = 'categorical_crossentropy',
    metrics=['accuracy']
)

### 모델 학습  

steps_per_epoch증가시킴 - 이미지 부풀리기를 통해 100개

In [15]:
model.fit(
        train_generator,
        steps_per_epoch=15,                   
        epochs=200,
        verbose=2,
        validation_data=test_generator,
        validation_steps=5)

Epoch 1/200
15/15 - 0s - loss: 0.9904 - accuracy: 0.5556 - val_loss: 0.9933 - val_accuracy: 0.5333
Epoch 2/200
15/15 - 0s - loss: 0.8634 - accuracy: 0.5556 - val_loss: 1.0053 - val_accuracy: 0.6000
Epoch 3/200
15/15 - 0s - loss: 0.7485 - accuracy: 0.7111 - val_loss: 1.2401 - val_accuracy: 0.4000
Epoch 4/200
15/15 - 0s - loss: 0.5775 - accuracy: 0.7333 - val_loss: 1.3203 - val_accuracy: 0.5333
Epoch 5/200
15/15 - 0s - loss: 0.4757 - accuracy: 0.7778 - val_loss: 1.8508 - val_accuracy: 0.5333
Epoch 6/200
15/15 - 0s - loss: 0.4455 - accuracy: 0.7778 - val_loss: 1.6244 - val_accuracy: 0.4667
Epoch 7/200
15/15 - 0s - loss: 0.3125 - accuracy: 0.8889 - val_loss: 2.0896 - val_accuracy: 0.5333
Epoch 8/200
15/15 - 0s - loss: 0.5478 - accuracy: 0.7778 - val_loss: 2.0889 - val_accuracy: 0.5333
Epoch 9/200
15/15 - 0s - loss: 0.2652 - accuracy: 0.8889 - val_loss: 1.8153 - val_accuracy: 0.4667
Epoch 10/200
15/15 - 0s - loss: 0.2643 - accuracy: 0.9111 - val_loss: 1.6146 - val_accuracy: 0.6000
Epoch 11/

### 모델 평가

In [16]:
model.evaluate(test_generator, steps=5)

5/5 [==============================] - 0s 2ms/step - loss: 2.7955 - accuracy: 0.7333


[2.7954771518707275, 0.7333333492279053]